<a href="https://colab.research.google.com/github/Gus-1003/ISD-invention/blob/main/Plots/2D_Heatmap_Massivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive

drive.mount('/content/drive/')

%cd /content/drive/MyDrive/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive


In [29]:
import plotly.express as px
import pandas as pd # manipulação e análise de dados
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import glob
import cv2

from scipy.ndimage import gaussian_filter

In [38]:
# Diretório onde estão os arquivos .csv
diretorio = "/content/drive/Shareddrives/Videos OPTO/Videos_DLC"  # Substitua pelo diretório correto

# Use a função glob para listar todos os arquivos .csv no diretório
arquivos_csv = glob.glob(f"{diretorio}/*.csv")

# Use a função glob para listar todos os arquivos . no diretório
arquivos_mp4 = glob.glob(f"{diretorio}/*.mp4")

print('\n'.join(arquivos_csv))

/content/drive/Shareddrives/Videos OPTO/Videos_DLC/MTOP4-TEST-E1DLC_resnet50_Videos OPTOSep1shuffle1_182500.csv
/content/drive/Shareddrives/Videos OPTO/Videos_DLC/MTOP4-TR-E3DLC_resnet50_Videos OPTOSep1shuffle1_182500.csv
/content/drive/Shareddrives/Videos OPTO/Videos_DLC/MTOP4-TEST-E3DLC_resnet50_Videos OPTOSep1shuffle1_182500.csv
/content/drive/Shareddrives/Videos OPTO/Videos_DLC/MTOP5-TR-E1DLC_resnet50_Videos OPTOSep1shuffle1_182500.csv
/content/drive/Shareddrives/Videos OPTO/Videos_DLC/MTOP5-TEST-E1DLC_resnet50_Videos OPTOSep1shuffle1_182500.csv
/content/drive/Shareddrives/Videos OPTO/Videos_DLC/MTOP5-TEST2-E1DLC_resnet50_Videos OPTOSep1shuffle1_182500.csv
/content/drive/Shareddrives/Videos OPTO/Videos_DLC/MTOP7-TR-E3DLC_resnet50_Videos OPTOSep1shuffle1_182500.csv
/content/drive/Shareddrives/Videos OPTO/Videos_DLC/MTOP7-TEST-E3DLC_resnet50_Videos OPTOSep1shuffle1_182500.csv
/content/drive/Shareddrives/Videos OPTO/Videos_DLC/MTOP11-TR-E1DLC_resnet50_Videos OPTOSep1shuffle1_182500.cs

In [26]:
# Lista para armazenar os DataFrames lidos de cada arquivo
dataframes = []

# Crie uma figura para os plots
plt.figure(figsize=(10, 8))

# Loop para ler cada arquivo .csv e adicionar o DataFrame à lista
for arquivo in arquivos_csv:

  df = pd.read_csv(arquivo)

  # Para excluir as três primeiras colunas (A, B e C)
  df = df.iloc[:, 1:]

  # Concatene as linhas 0 e 1
  concatenated_row = df.iloc[0] + "_" + df.iloc[1]
  df.iloc[0] = concatenated_row

  # Substituir os nomes das colunas pelo conteúdo da primeira linha
  df.columns = df.iloc[0]

  # Remover a primeira linha (índice 0)
  df = df[2:]

  # Reset o índice
  df.reset_index(drop=True, inplace=True)

  bodyparts = "tail_base"
  bodyparts_x = "tail_base_x"
  bodyparts_y = "tail_base_y"

  # Converta as colunas center_x e center_y para valores numéricos
  df[bodyparts_x] = pd.to_numeric(df[bodyparts_x])
  df[bodyparts_y] = pd.to_numeric(df[bodyparts_y])

  # Normalize as coordenadas para o intervalo [0, 1]
  df[bodyparts_x] = (df[bodyparts_x] - df[bodyparts_x].min()) / (df[bodyparts_x].max() - df[bodyparts_x].min())
  df[bodyparts_y] = (df[bodyparts_y] - df[bodyparts_y].min()) / (df[bodyparts_y].max() - df[bodyparts_y].min())

  # Calcule o histograma 2D com interpolação
  x = df[bodyparts_x]
  y = df[bodyparts_y]
  heatmap, xedges, yedges = np.histogram2d(x, y, bins=(50, 50), density=True)

  df_teste = df[[bodyparts_x, bodyparts_y]]

  # Transponha o heatmap para corrigir a orientação
  heatmap = heatmap.T

  # Aplicar um filtro gaussiano para suavização
  heatmap_smoothed = gaussian_filter(heatmap, sigma=3)

  # Crie um gráfico de calor suavizado 2D com limite de valor máximo
  plt.imshow(heatmap_smoothed, cmap='viridis', extent=[x.min(), x.max(), y.min(), y.max()], aspect='auto', origin='lower', vmax=7)
  plt.colorbar()

  # Configurar os rótulos dos eixos
  plt.xlabel('x')
  plt.ylabel('y')

  # Obtenha o nome do arquivo sem a extensão
  nome_arquivo = os.path.splitext(os.path.basename(arquivo))[0]

  # Crie uma pasta de destino com o nome do arquivo (se não existir)
  pasta_destino = f"/content/drive/Shareddrives/Videos OPTO/Plots_Videos_DLC/{nome_arquivo}"
  os.makedirs(pasta_destino, exist_ok=True)

  # Caminho completo para o arquivo de imagem a ser salvo
  caminho_arquivo_salvo = os.path.join(pasta_destino, f"{nome_arquivo}_{bodyparts}_heatmap.png")

  # Salve o gráfico na pasta de destino
  plt.savefig(caminho_arquivo_salvo)

  # Feche o gráfico para liberar memória
  plt.close()

In [40]:
# Lista para armazenar o número "x"
images = []

# Loop para processar cada vídeo
for video in arquivos_mp4:

  # Verificar se o caminho do vídeo existe
  if not os.path.exists(video):
      print(f"O vídeo '{video}' existe.")
      continue

  # Abrir o vídeo
  vidcap = cv2.VideoCapture(video)

  # Verificar se o vídeo foi aberto corretamente
  if not vidcap.isOpened():
      print(f"Falha ao abrir o vídeo '{video}'.")
      continue

  # Obtém o número total de frames do vídeo
  total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

  # Obtém o frame rate do vídeo (número de frames por segundo)
  frame_rate = vidcap.get(cv2.CAP_PROP_FPS)

  # Calcula a duração total do vídeo em segundos
  video_duration = total_frames / frame_rate

  # Calcula o intervalo de tempo entre cada frame com base no contador
  intervalo = video_duration / 2

  # Obtenha o nome do arquivo sem a extensão
  nome_arquivo = os.path.splitext(os.path.basename(video))[0]

  # Calcula o tempo do frame do meio
  frame_time = intervalo

  # Calcula o número do frame correspondente ao tempo calculado
  frame_number = int(frame_time * frame_rate)

  # Define o número do frame atual no vídeo
  vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

  # Lê o frame correspondente
  ret, frame = vidcap.read()

  # Verifica se a leitura do frame foi bem-sucedida
  if ret:
      # Crie uma pasta de destino com o nome do arquivo (se não existir)
      pasta_destino = f"/content/drive/Shareddrives/Videos OPTO/Plots_Videos_DLC/{nome_arquivo}"
      os.makedirs(pasta_destino, exist_ok=True)

      # Caminho completo para o arquivo de imagem a ser salvo
      caminho_arquivo_salvo = os.path.join(pasta_destino, f"{nome_arquivo}_frame.png")
      print({caminho_arquivo_salvo})

      # Salvar o frame do meio como uma imagem
      cv2.imwrite(caminho_arquivo_salvo, frame)

  # Feche o vídeo
  vidcap.release()

{'/content/drive/Shareddrives/Videos OPTO/Plots_Videos_DLC/MTOP4-TEST-E1DLC_resnet50_Videos OPTOSep1shuffle1_182500_labeled/MTOP4-TEST-E1DLC_resnet50_Videos OPTOSep1shuffle1_182500_labeled_frame.png'}
{'/content/drive/Shareddrives/Videos OPTO/Plots_Videos_DLC/MTOP5-TEST2-E1DLC_resnet50_Videos OPTOSep1shuffle1_182500_labeled/MTOP5-TEST2-E1DLC_resnet50_Videos OPTOSep1shuffle1_182500_labeled_frame.png'}
{'/content/drive/Shareddrives/Videos OPTO/Plots_Videos_DLC/MTOP7-TR-E3DLC_resnet50_Videos OPTOSep1shuffle1_182500_labeled/MTOP7-TR-E3DLC_resnet50_Videos OPTOSep1shuffle1_182500_labeled_frame.png'}
{'/content/drive/Shareddrives/Videos OPTO/Plots_Videos_DLC/MTOP4-TR-E3DLC_resnet50_Videos OPTOSep1shuffle1_182500_labeled/MTOP4-TR-E3DLC_resnet50_Videos OPTOSep1shuffle1_182500_labeled_frame.png'}
{'/content/drive/Shareddrives/Videos OPTO/Plots_Videos_DLC/MTOP7-TEST-E3DLC_resnet50_Videos OPTOSep1shuffle1_182500_labeled/MTOP7-TEST-E3DLC_resnet50_Videos OPTOSep1shuffle1_182500_labeled_frame.png'}
{